In [47]:
#性能测量
def meansure_performance(y_predict_lis):
    TP=0
    FP=0
    TN=0
    FN=0
    #测试样本数
    sample_num=x_test.shape[0]
    for i in range(sample_num):
        y_predict=y_predict_lis[i]
        if y_predict==1 and y_test[i]==1:
            TP+=1
        elif y_predict==-1 and y_test[i]==-1:
            TN+=1
        elif y_predict==1 and y_test[i]==-1:
    #false poistive
            FP+=1
        elif y_predict==-1 and y_test[i]==1:
    #false negative
            FN+=1 
    accuracy=(TP+TN)/(TP+FP+TN+FN)
    recall=TP/(TP+FN)
    specificity=TN/(TN+FP)
    precision=TP/(TP+FP)
    F_measuer=(2*precision*recall)/(precision+recall)
    return accuracy,recall,specificity,precision,F_measuer

In [48]:
from skfeature.function.similarity_based import reliefF
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_yanghe.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

In [49]:
techini_indicator=data_rfe.columns.tolist()[:-1]

# SVC

In [50]:
estimator_SVC=SVC(C=0.2, kernel='rbf',gamma=0.1)
estimator_SVC.fit(x_train,y_train)
SVC_result= estimator_SVC.predict(x_test)
performan_SVC=meansure_performance(SVC_result)
print(performan_SVC)

(0.6588235294117647, 0.8378378378378378, 0.5208333333333334, 0.5740740740740741, 0.6813186813186813)


 # CFS-SVM

In [51]:
CFS_selected_feature = CFS.cfs(x_train,y_train)
CFS_fea_sele=list(CFS_selected_feature)
CFS_fea_sele

[4, 26, 6, 1, 2, 3]

In [52]:
np.array(techini_indicator)[CFS_fea_sele],len(np.array(techini_indicator)[CFS_fea_sele])

(array(['VOSC', 'K', 'CR', 'TAPI', 'VMA', 'VMACD'], dtype='<U8'), 6)

In [53]:
estimator_SVM_CFS=SVC(C=0.2, kernel='rbf',gamma=0.1)
estimator_SVM_CFS.fit(x_train[:,CFS_fea_sele],y_train)
CFS_SVM_result=estimator_SVM_CFS.predict(x_test[:,CFS_fea_sele])

In [54]:
performan_CFS_SVM=meansure_performance(CFS_SVM_result)
print(performan_CFS_SVM)

(0.6058823529411764, 0.47297297297297297, 0.7083333333333334, 0.5555555555555556, 0.5109489051094891)


# sequencial-backward-elimination SVM

In [55]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class back_SVM:
    def __init__(self,X_train,Y_train,c,gamma):
        self.X_train=X_train
        self.Y_train=Y_train
        self.C=c
        self.Gamma=gamma
    def SBE(self,folds):
        fea_num=self.X_train.shape[1]
        candi_fea=[i for i in range(fea_num)]
        fea=[i for i in range(fea_num)]
        best_subset=[i for i in range(fea_num)]
        model=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        Best_accuracy=mean(cross_val_score(model,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        Success=True
        while Success:
            Success=False
            for f in candi_fea:
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(f)
                x_sele=self.X_train[:,candi_fea_copy]
                n_scores=cross_val_score(model,x_sele,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1)
                acc=mean(n_scores)
                if acc>=Best_accuracy:
                    #print(f)
                    best_subset=candi_fea_copy
                    Success=True
                    Best_accuracy=acc
            print(1)
            if Success==True:
                candi_fea=best_subset
            print(len(candi_fea))
        return best_subset

In [56]:
estimator_SVM_back=back_SVM(x_train,y_train,0.2,0.1)
candi_fea_SBE=estimator_SVM_back.SBE(5)

1
48
1
47
1
46
1
45
1
44
1
43
1
42
1
41
1
40
1
39
1
38
1
37
1
36
1
35
1
34
1
33
1
32
1
31
1
30
1
29
1
28
1
27
1
26
1
25
1
24
1
23
1
22
1
21
1
20
1
19
1
19


In [57]:
np.array(techini_indicator)[candi_fea_SBE],len(np.array(techini_indicator)[candi_fea_SBE])

(array(['QRR', 'CR', 'PSY', 'AR', 'BR', 'MOBV', 'TRIX', 'BIAS', 'DBCD',
        'K', 'D', 'J', 'RSI', 'VROC', 'VRSI', 'WR', 'ADTM', 'SRMI', 'CVLT'],
       dtype='<U8'),
 19)

In [58]:
estimator_SVM_SBE=SVC(C=0.2, kernel='rbf',gamma=0.1)
estimator_SVM_SBE.fit(x_train[:,candi_fea_SBE],y_train)
SBE_SVM_result=estimator_SVM_SBE.predict(x_test[:,candi_fea_SBE])
performan_SBE_SVM=meansure_performance(SBE_SVM_result)
print(performan_SBE_SVM)

(0.7647058823529411, 0.7162162162162162, 0.8020833333333334, 0.7361111111111112, 0.7260273972602739)


# reliefF-svm 

In [59]:
from skfeature.function.similarity_based import reliefF
relief_fea_score=reliefF.reliefF(x_train,y_train)
relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))[:1]

In [60]:
relief_fea_score

array([ 0.05932584,  0.03220252, -0.00809487, -0.00291175, -0.01488879,
       -0.00892114, -0.01041739,  0.01269864, -0.00451808,  0.0044056 ,
       -0.00193376, -0.00597814,  0.52760681, -0.00784431,  0.00721879,
       -0.00367675, -0.00260998, -0.00464684, -0.01107858,  0.0067129 ,
        0.00307179,  0.00317558,  0.02886583, -0.00556633, -0.00727733,
        0.02858033, -0.01907913,  0.0025571 , -0.01240215,  0.07704528,
        0.00299734,  0.00874352,  0.08149064,  0.067765  , -0.0023986 ,
        0.00778985, -0.00988366, -0.0034835 , -0.00408301,  0.00218345,
        0.00802034, -0.00831424, -0.00507982, -0.00805214, -0.00979869,
        0.001794  , -0.01652236, -0.00624788, -0.0206097 ])

In [61]:
np.array(techini_indicator)[relief_candi_fea],len(np.array(techini_indicator)[relief_candi_fea])

(array(['MOBV'], dtype='<U8'), 1)

In [62]:
estimator_SVM_relief=SVC(C=0.2, kernel='rbf',gamma=0.1)
estimator_SVM_relief.fit(x_train[:,relief_candi_fea],y_train)
relief_SVM_result=estimator_SVM_relief.predict(x_test[:,relief_candi_fea])
performan_relief_SVM=meansure_performance(relief_SVM_result)
print(performan_relief_SVM)

(0.7764705882352941, 0.7972972972972973, 0.7604166666666666, 0.7195121951219512, 0.7564102564102565)


# RFE-SVM

In [63]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,folds):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            if len(candi_fea)==featur_num:
                return candi_fea
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                #print(scor_fea)
                rfe_score[k]=scor_fea
                k=k+1     
            #print(rfe_score)
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(1)

In [64]:
rfe_SVM_model=rfe_SVM(x_train,y_train,0.2,0.1)
rfe_SVM_sele_featu_subset=rfe_SVM_model.featu_subset(1)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [65]:
np.array(techini_indicator)[rfe_SVM_sele_featu_subset],len(np.array(techini_indicator)[rfe_SVM_sele_featu_subset])

(array(['MOBV'], dtype='<U8'), 1)

In [66]:
estimator_SVM_rfe=SVC(C=0.2, kernel='rbf',gamma=0.1)
estimator_SVM_rfe.fit(x_train[:,rfe_SVM_sele_featu_subset],y_train)
rfe_SVM_result=estimator_SVM_rfe.predict(x_test[:,rfe_SVM_sele_featu_subset])
performan_rfe_SVM=meansure_performance(rfe_SVM_result)
print(performan_rfe_SVM)

(0.7764705882352941, 0.7972972972972973, 0.7604166666666666, 0.7195121951219512, 0.7564102564102565)


# SVM-RFE_RELIEF

In [67]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,alpha,folds):
        #kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            if len(candi_fea)==optim_featur_num:
                return candi_fea
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(i)
            i-=1
            

In [68]:
# k=4# k=5, 0.8
rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.2,0.1)
rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.95,1)

49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2


In [69]:
np.array(techini_indicator)[rfe_relief_SVM_sele_featu_subset],len(np.array(techini_indicator)[rfe_relief_SVM_sele_featu_subset])

(array(['MOBV'], dtype='<U8'), 1)

In [70]:
estimator_SVM_rfe_reliefF=SVC(C=0.2, kernel='rbf',gamma=0.1)
estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
print(performan_rfe_reliefF_SVM)

(0.7764705882352941, 0.7972972972972973, 0.7604166666666666, 0.7195121951219512, 0.7564102564102565)
